In [ ]:
import noisereduce as nr
import librosa
import soundfile as sf

# Load audio file
y, sr = librosa.load("/content/drive/MyDrive/AI internship/Audio_files/2/05599ADA-6B7A-426A-81CE-2725BEB26D94.wav", sr=None)

# Perform noise reduction
reduced_noise = nr.reduce_noise(y=y, sr=sr)

# Save the enhanced audio to a new file
sf.write("/content/drive/MyDrive/AI internship/Audio_files/Noise_Removal_Files/denoised.wav", reduced_noise, sr)

In [ ]:
#wavelet based Algorithm
pip install PyWavelets numpy matplotlib soundfile
pip show PyWavelets numpy matplotlib soundfile
import pywt
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf

# Load the noisy audio signal
input_audio_file = "/content/drive/MyDrive/Category 2/B2A4D737-AFBD-44AA-B434-88E9568C000D.wav"
noisy_signal, sample_rate = sf.read(input_audio_file)

# Define the wavelet and level of decomposition
wavelet = 'db4'  # Choose a wavelet, e.g., 'db4'
level = 5  # Level of decomposition (adjust as needed)

# Perform wavelet decomposition
coeffs = pywt.wavedec(noisy_signal, wavelet, level=level)

# Threshold the detail coefficients (soft or hard thresholding)
threshold = 0.01  # Adjust the threshold as needed
coeffs_thresh = [pywt.threshold(detail, threshold, mode='soft') for detail in coeffs[1:]]

# Reconstruct the denoised signal
denoised_signal = pywt.waverec([coeffs[0]] + coeffs_thresh, wavelet)

# Save the denoised audio to an output file with a .wav extension
output_audio_file = "/content/drive/MyDrive/combined_output.wav"
sf.write(output_audio_file, denoised_signal, sample_rate)

# Plot and display the original and denoised signals (optional)
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.title("Noisy Signal")
plt.plot(noisy_signal)
plt.subplot(2, 1, 2)
plt.title("Denoised Signal")
plt.plot(denoised_signal)
plt.tight_layout()
plt.show()


In [ ]:
#Nomalized LMS filter
import numpy as np
from scipy.io import wavfile
from scipy.signal import lfilter

def normalized_lms_filter(signal, mu=0.5, N=64):
    w = np.zeros(N)  # Initialize the filter weights
    output = np.zeros_like(signal)  # Initialize the output signal

    for i in range(N, len(signal)):
        x = signal[i-N:i]  # Extract the input vector
        y = np.dot(w, x)  # Compute the output of the adaptive filter
        e = signal[i] - y  # Compute the error signal
        norm_x = np.dot(x, x) + 1e-6  # Avoid division by zero
        w += (mu / norm_x) * e * x  # Update the filter weights using the NLMS algorithm
        output[i] = y  # Store the output of the adaptive filter

    return output

# Load the uploaded audio file
file_path = '/path/to/your/audio/file.wav'  # Replace with the path to your audio file
sr, audio = wavfile.read(file_path)
audio = audio.astype(np.float32) / np.iinfo(np.int16).max  # Assuming the original data type is int16

# Apply the NLMS adaptive filter
filtered_audio = normalized_lms_filter(audio)

# Save the filtered audio to a new file
output_file_path = '/path/to/your/output/file.wav'  # Replace with the path where you want to save the output file
wavfile.write(output_file_path, sr, (filtered_audio * np.iinfo(np.int16).max).astype(np.int16))

In [ ]:
#Wiener Filter
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import stft, istft
from scipy.io.wavfile import write

k2# Function to normalize audio
def normalize_audio(input_signal, target_rms=0.1):
    current_rms = np.sqrt(np.mean(np.square(input_signal)))
    if current_rms == 0:
        return input_signal
    factor = target_rms / current_rms
    normalized_audio = input_signal * factor
    return normalized_audio

# Function to perform noise reduction using Wiener filtering
def wiener_filter_noise_reduction(signal, noise_segment, n_fft=2048, hop_length=512):
    _, _, Zxx = stft(signal, nperseg=n_fft, noverlap=hop_length)
    _, _, Zxx_noise = stft(noise_segment, nperseg=n_fft, noverlap=hop_length)
    noise_psd = np.mean(np.abs(Zxx_noise) ** 2, axis=1)
    signal_psd = np.abs(Zxx) ** 2
    wiener_filter = signal_psd / (signal_psd + noise_psd[:, np.newaxis] + 1e-10)
    Zxx_reduced = wiener_filter * Zxx
    _, reduced_signal = istft(Zxx_reduced, noverlap=hop_length)
    reduced_signal *= np.linalg.norm(signal) / np.linalg.norm(reduced_signal)
    return reduced_signal

# Load the audio file from Google Drive
file_path = '/content/drive/YourPath/0A534BFB-8002-418E-B3AA-661E5EB369AC.wav' # Adjust your file path
audio, sr = librosa.load(file_path, sr=None)
noise_sample = audio[:int(0.5 * sr)] # You can adjust the duration for the noise sample as per your requirement

# Apply Wiener filter noise reduction
denoised_audio = wiener_filter_noise_reduction(audio, noise_sample)

# Plot the waveform of the denoised audio
plt.figure(figsize=(10, 4))
librosa.display.waveshow(denoised_audio, sr=sr)
plt.title('Waveform of Audio after Wiener Filter Noise Reduction')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.show()

# Save the denoised audio to a new file
output_file_path = '/content/drive/YourPath/denoised_audio.wav' # Adjust your output file path
librosa.output.write_wav(output_file_path, denoised_audio, sr)

In [ ]:
pip install speechbrain

In [ ]:
#Short-Time Fourier Transform (STFT)
import numpy as np
import librosa
from scipy.signal import stft, istft
import soundfile as sf

# Load the audio file
file_path_advanced = '/path/to/your/audio/file.wav'  # Replace with the path to your audio file
audio_advanced, sr_advanced = librosa.load(file_path_advanced, sr=None)

# Define the frame length and hop length for the STFT
frame_length_advanced = int(sr_advanced * 0.025)  # 25 ms frames
hop_length_advanced = frame_length_advanced // 2  # 50% overlap

# Compute the STFT of the audio signal
f_advanced, t_advanced, Zxx_advanced = stft(audio_advanced, fs=sr_advanced, nperseg=frame_length_advanced, noverlap=hop_length_advanced)

# Initialize variables to store the noise estimate and the noise mask
noise_est_advanced = np.zeros_like(Zxx_advanced)
noise_mask_advanced = np.ones_like(Zxx_advanced)

# Define the number of frames to use for the initial noise estimate and the alpha parameter for noise estimation
init_frames_advanced = 5  # Use the first 5 frames to initialize the noise estimate
alpha_advanced = 0.9  # Smoothing factor for noise estimation

# Initialize the noise estimate using the first few frames of the audio signal
noise_est_advanced[:, :init_frames_advanced] = np.abs(Zxx_advanced[:, :init_frames_advanced])

# Iterate over each frame and update the noise estimate and compute the noise mask
for i in range(init_frames_advanced, Zxx_advanced.shape[1]):
    # Update the noise estimate using a first-order recursive filter
    noise_est_advanced[:, i] = alpha_advanced * noise_est_advanced[:, i - 1] + (1 - alpha_advanced) * np.abs(Zxx_advanced[:, i])

    # Compute the noise mask based on the updated noise estimate
    noise_mask_advanced[:, i] = 1 - (noise_est_advanced[:, i] / (np.abs(Zxx_advanced[:, i]) + 1e-10))

# Apply the noise mask to the noisy STFT to obtain the clean STFT
Zxx_clean_advanced = noise_mask_advanced * Zxx_advanced

# Perform the inverse STFT to obtain the time-domain signal of the clean audio
_, audio_clean_advanced = istft(Zxx_clean_advanced, fs=sr_advanced, noverlap=hop_length_advanced)

# Normalize the amplitude of the clean audio
audio_clean_advanced *= np.linalg.norm(audio_advanced) / np.linalg.norm(audio_clean_advanced)

# Save the cleaned audio to a new file
output_file_path_advanced = '/path/to/your/output/audio/file.wav'  # Replace with the path where you want to save the output file
sf.write(output_file_path_advanced, audio_clean_advanced, sr_advanced)

In [ ]:
#Root Mean Square(RMS Level)
import noisereduce as nr
import librosa
import numpy as np
import soundfile as sf

# Load the audio file
file_path = '/content/drive/MyDrive/2/71A30F1B-9E3C-42A6-98C4-FD04A8E188EA.wav'  # Replace with the path to your audio file
audio, sr = librosa.load(file_path, sr=None)

# Amplify the noise (You can adjust the amplification factor as needed)
amplification_factor = 1.7
amplified_audio = audio * amplification_factor

# Perform noise reduction on the amplified noise
denoised_audio = nr.reduce_noise(sr=sr, y=amplified_audio)

# Normalize the denoised audio to the same RMS level as the original audio
rms_original = np.sqrt(np.mean(audio**2))
rms_denoised = np.sqrt(np.mean(denoised_audio**2))
normalized_audio = denoised_audio * (rms_original / rms_denoised)

# Save the normalized denoised audio to a new file
output_file_path = '/content/drive/MyDrive/Audio_out/file88EA.wav'  # Replace with the path where you want to save the output file
sf.write(output_file_path, normalized_audio, sr)

In [ ]:
#Speech Brain
from speechbrain.pretrained import SepformerSeparation as separator
import torchaudio

# Initialize the separator model from SpeechBrain
model = separator.from_hparams(
    source="speechbrain/sepformer-dns4-16k-enhancement",
    savedir='tmpdir/sepformer-dns4-16k-enhancement'
)

# Define the paths
input_file_path = '/mnt/data/0A534BFB-8002-418E-B3AA-661E5EB369AC.wav'
resampled_file_path = '/mnt/data/resampled_0A534BFB-8002-418E-B3AA-661E5EB369AC.wav'
output_file_path = '/mnt/data/enhanced_dns4-16k_0A534BFB-8002-418E-B3AA-661E5EB369AC.wav'

# Load and resample the audio file to 16000 Hz
audio, _ = librosa.load(input_file_path, sr=16000)
sf.write(resampled_file_path, audio, 16000)

# Apply the model
est_sources = model.separate_file(path=resampled_file_path)

# Save the separated source to a new file
torchaudio.save(output_file_path, est_sources[:, :, 0].detach().cpu(), 16000)

# Provide the path to the processed file
output_file_path

In [ ]:
# working----
import torchaudio
from speechbrain.pretrained import WaveformEnhancement

enhance_model = WaveformEnhancement.from_hparams(
    source="speechbrain/mtl-mimic-voicebank",
    savedir="pretrained_models/mtl-mimic-voicebank",
)
enhanced = enhance_model.enhance_file("/content/0A534BFB-8002-418E-B3AA-661E5EB369AC.wav")

# Saving enhanced signal on disk
torchaudio.save('/content/0A534BFB-8002-418E-B3AA-661E5EB369AC_enhanced.wav', enhanced.unsqueeze(0).cpu(), 16000)

In [ ]:
# working----
import torch
from speechbrain.pretrained import SpectralMaskEnhancement

# Initialize the SpectralMaskEnhancement model
enhancer = SpectralMaskEnhancement.from_hparams(
    source="speechbrain/metricgan-plus-voicebank"
)

# Enhance an audio file
enhanced = enhancer.enhance_file(
    "/content/test_audio.wav"
)
# Saving enhanced signal on disk
torchaudio.save('/content/test_audio_enhanced_audio.wav', enhanced.unsqueeze(0).cpu(), 16000)

In [ ]:
!pip install noisereduce

import noisereduce as nr
import librosa
import numpy as np
import soundfile as sf


# Load the audio file
file_path = '/content/0A534BFB-8002-418E-B3AA-661E5EB369AC.wav'  # Replace with the path to your audio file
audio, sr = librosa.load(file_path, sr=None)

# Perform noise reduction
denoised_audio = nr.reduce_noise(sr=sr, y=audio)


# Normalize the denoised audio to the same RMS level as the original audio
rms_original = np.sqrt(np.mean(audio**2))
rms_denoised = np.sqrt(np.mean(denoised_audio**2))
normalized_audio = denoised_audio * (rms_original / rms_denoised)

# Save the normalized denoised audio to a new file
output_file_path = '/content/file1.wav'  # Replace with the path where you want to save the output file
sf.write(output_file_path, normalized_audio, sr)

In [ ]:
#Conservative spectral Gatting

# Re-importing necessary libraries and redefining the functions
import librosa
import numpy as np
from scipy.signal import stft, istft
from scipy.io.wavfile import write


# Redefining the noise reduction and normalization functions
def conservative_spectral_gating_noise_reduction(signal, noise_segment, n_fft=2048, hop_length=512, alpha=0.3):
    _, _, Zxx = stft(signal, nperseg=n_fft, noverlap=hop_length)
    _, _, Zxx_noise = stft(noise_segment, nperseg=n_fft, noverlap=hop_length)
    noise_mean = np.mean(np.abs(Zxx_noise), axis=1)
    gain = np.maximum(1 - alpha * (noise_mean[:, np.newaxis] / (np.abs(Zxx) + 1e-10)), 0)
    Zxx_reduced = gain * Zxx
    _, reduced_signal = istft(Zxx_reduced, noverlap=hop_length)
    reduced_signal *= np.linalg.norm(signal) / np.linalg.norm(reduced_signal)
    return reduced_signal


def normalize_audio(input_signal, target_rms=0.1):
    current_rms = np.sqrt(np.mean(np.square(input_signal)))
    if current_rms == 0:
        return input_signal
    factor = target_rms / current_rms
    normalized_audio = input_signal * factor
    return normalized_audio


# Load the uploaded file again
file_path = '/content/0A534BFB-8002-418E-B3AA-661E5EB369AC.wav'
signal, sr = librosa.load(file_path, sr=None)

# Assume the first second of the audio is representative of the noise in the file.
noise_segment = signal[:sr]  # First second for noise

# Apply the conservative spectral gating noise reduction method again
conservative_spectral_reduced_audio = conservative_spectral_gating_noise_reduction(signal, noise_segment)

# Normalize the amplitude of the reduced audio again
conservative_spectral_normalized_audio = normalize_audio(conservative_spectral_reduced_audio)

# Save the processed audio to a new file again
conservative_processed_file_path = '/content/conservative_processed_spectral_0A534BFB-8002-418E-B3AA-661E5EB369AC.wav'
write(conservative_processed_file_path, sr, conservative_spectral_normalized_audio.astype(np.float32))

# Provide the path to the processed file
conservative_processed_file_path

In [ ]:
#Adaptive Noise Reduction
import numpy as np
import librosa
import soundfile as sf

# Load the audio file
audio_file_path = "/content/drive/MyDrive/AI internship/Audio_files/1/0699CB4D-B694-4922-ABAD-3E50C67D6C19.wav"
y, sr = librosa.load(audio_file_path, sr=None)

# ---- Adaptive Noise Reduction using Spectral Gating ----
def spectral_gating(y, sr, noise_reduction_factor=1.5):
    S_full, phase = librosa.magphase(librosa.stft(y))
    S_filter = librosa.decompose.nn_filter(S_full,
                                           aggregate=np.median,
                                           metric='cosine',)
    S_filter = np.minimum(S_full, S_filter)
    margin_i, margin_v = 2, 10
    power = 2

    mask_i = librosa.util.softmask(S_filter,
                                   margin_i * (S_full - S_filter),
                                   power=power)

    mask_v = librosa.util.softmask(S_full - S_filter,
                                   margin_v * S_filter,
                                   power=power)

    S_foreground = mask_v * S_full
    y_denoised = librosa.istft(S_foreground * phase)
    return y_denoised

y_denoised = spectral_gating(y, sr)

# ---- Selective Amplification ----
def selective_amplification(y, frame_length=1024, hop_length=512, gain_factor=1.2):
    # Divide the audio into frames
    frames = librosa.util.frame(y, frame_length=frame_length, hop_length=hop_length).T

    # Initialize an empty array to store the amplified audio
    amplified_audio = np.array([])

    # Loop through each frame
    for frame in frames:
        # Calculate the amplitude of this frame
        amplitude = np.mean(np.abs(frame))

        # If the amplitude is below a certain threshold, amplify it
        if amplitude < 0.05:  # Threshold can be adjusted
            frame = frame * gain_factor

        # Append the frame to the amplified_audio array
        amplified_audio = np.concatenate([amplified_audio, frame])

    return amplified_audio

y_amplified = selective_amplification(y_denoised)

# ---- Save the processed audio ----
sf.write("adaptive_noise_reduced_audio.wav", y_denoised, sr)
sf.write("selective_amplified_audio.wav", y_amplified, sr